### **Preprocessing**
**# Preprocess 'utterance' column**
- Step 1. Tokenization -> split()
- Step 2. Word Embedding -> pretrained Glove
- Step 3. Create 'Vectorization' column
- Step 4. Save .csv file

#### Sentence Vectorization

In [2]:
import pandas as pd
import numpy as np

# GloVe 임베딩 로드
def load_glove_model(glove_file_path):
    glove_model = {}
    with open(glove_file_path, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            vector = np.array(values[1:], dtype='float32')
            glove_model[word] = vector
    return glove_model

# 문장을 단어로 토큰화하여 GloVe 벡터로 변환하는 함수 정의
def sentence_to_vector(sentence, glove_model):
    tokens = sentence.split()
    vectorized_tokens = [glove_model[token.lower()] for token in tokens if token.lower() in glove_model]
    if vectorized_tokens:
        # 각 단어 벡터를 평균하여 문장 벡터 생성
        sentence_vector = np.mean(vectorized_tokens, axis=0)
    else:
        # 단어 벡터가 없는 경우 기본적인 임베딩 값 반환
        sentence_vector = np.zeros_like(next(iter(glove_model.values())))
    return sentence_vector

# CSV 파일 경로
csv_file_path = "/content/drive/MyDrive/2024-1 graph-mining/final_project/sample_cont_280.csv"

# GloVe 파일 경로
glove_file_path = "/content/drive/MyDrive/2024-1 graph-mining/final_project/glove.6B.300d.txt"
glove_model = load_glove_model(glove_file_path)

# CSV 파일 읽기
data = pd.read_csv(csv_file_path)

# 'utterance' 열의 각 문장을 GloVe를 사용하여 토큰화하고 벡터화하여 'Vectorization' 열에 저장
data['Vectorization'] = data['utterance'].apply(lambda x: sentence_to_vector(x, glove_model))

# 결과 출력
print(data.head())

# 수정된 데이터프레임을 CSV 파일로 저장
output_csv_file_path = "/content/drive/MyDrive/2024-1 graph-mining/final_project/Contrastive_with_vectors_280.csv"
data.to_csv(output_csv_file_path, index=False)

       emotion                                          utterance  \
0  contentment  this reminds me of my house as a child growing up   
1      sadness  The face and body of the person in painting lo...   
2      sadness                      This lady looks sad and lost.   
3         fear  there is only really one line in the picture a...   
4        anger  The woman will not allow black people to enter...   

              art_style                             painting  repetition  \
0               Realism                        do not called          13   
1         Impressionism                      boy in sea foam           9   
2         Expressionism       germaine survage with earrings          10   
3  Color_Field_Painting                            the voice          15   
4         Impressionism  helene rouart in her father s study           4   

               artist  year                                      Vectorization  
0  pyotr konchalovsky  1947  [-0.23660432, 0.02

#### Calculate Cosine Similarity

In [3]:
from numpy import dot
from numpy.linalg import norm

def cosine_similarity(vec1, vec2):
    if norm(vec1) > 0 and norm(vec2) > 0:
        return dot(vec1, vec2) / (norm(vec1) * norm(vec2))
    else:
        return 0  # 두 벡터 중 하나라도 영 벡터인 경우 유사도는 0

# 첫 번째 문장과 두 번째 문장의 벡터 가져오기
vector1 = data['Vectorization'][0]
vector2 = data['Vectorization'][1]

# 코사인 유사도 계산
similarity = cosine_similarity(vector1, vector2)
print("첫 번째 문장과 두 번째 문장의 코사인 유사도:", similarity)

첫 번째 문장과 두 번째 문장의 코사인 유사도: 0.9074012
